In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

https://campus.datacamp.com/courses/foundations-of-predictive-analytics-in-python-part-2

> part 1. choose the right data to use
  - timeline
  - population
  - target

> part 2. Adding evolutions
    - add trending: YoY metric to track change


In [ ]:
Import pandas as pd
gifts = pd.read_csv(“gifts.csv”)
gifts["date"] = pd.to_datetime(gifts["date"])
print(gifts.head())
#    id       date  amount
# 0   1 2015-10-16    75.0
# 1   1 2014-02-11   111.0
# 2   1 2012-03-28    93.0
# 3   1 2013-12-13   113.0
# 4   1 2012-01-10    93.0
start_target = datetime(year = 2018, month = 5, day = 1)
# reference_date = datetime.date(2018,4,1)
end_target = datetime(year = 2018, month = 8, day = 1)
gifts_target = gifts[(gifts["date"]>=start_target) & (gifts["date"]<end_target)]
gifts_pred_variables = gifts[(gifts["date"]<start_target]

In [ ]:
"""
population example: people who donate in 2016 but didn't donate in 2017 yet
"""
donations_2016 = gifts[gifts["date"].dt.year==2016]
donors_include = set(donations_2016["id"])
print(donors_include)
{1002,3043,4934, ...}
donations_2017 = gifts[(gifts["date"].dt.year==2017) 
                       & (gifts["date"].dt.month<5)]
donors_exclude = set(donations_2017["id"])
print(donors_exclude)
{2451,3047,4474, ...}
population = donors_include.difference(donors_exclude)

In [ ]:
# Reference date
reference_date = datetime.date(2018,4,1)
# Add lifetime to the basetable
basetable["lifetime"] = reference_date - basetable["member_since"]

"""
another example of age
Add a column "age" to the basetable that is the age of the donor on the reference date. 
"""

# Reference date
reference_date = datetime.date(2017, 5, 1)

# Add age to the basetable
basetable["age"] = (pd.Series([calculate_age(date_of_birth, reference_date)
                              for date_of_birth in basetable["date_of_birth"]]))

# Calculate mean age
print(round(basetable["age"].mean()))
# Count the number of donors with no segment assigned
print(basetable["segment"].isna().sum())


In [3]:
"""
Adding living place, but only use relevant data
"""
# Reference date
reference_date = datetime.date(2017, 5, 1)

# Select living place reference date
living_places_reference_date = living_places[(living_places["start_date"] <= reference_date) & 
                                            (living_places["end_date"] > reference_date)]

# Add living place to the basetable
basetable = pd.merge(basetable, living_places_reference_date[["donor_ID", "living_place"]], on="donor_ID")

> ### Part 3. data preparation
  - Creating dummies
     - prevent multicollinearity - remove one dummy
     - dummies_segment = pd.get_dummies(df["segment"],drop_first=True)
  - Missing values
     - imputation to median/mean
     - custom the value
  - Outliers: human/machine errors, extreme values
     - Winsorization concept: replace low 5/95 percentile to all 5/95 percentile value
     - Standard deviation method concept: mean+/- 3sd to replace low/high values
  - transformation
     - log tranformation: to know numeric difference in order of magnitude
     - interactions: one strategy is to include interactions of features of high importance on their own.
     - square foot transformation: np.sqrt
        - Observe that the differences in the column 'donations_sqrt' are smaller than in the column 'donations_log'. It depends on the situation which one to use, one option could be to add both to the predictive model and let the variable selection algorithm decide which one to use.

- example get dummy variable

In [ ]:
# Create the dummy variable
dummies_country = pd.get_dummies(basetable["country"], drop_first=True)

# Add the dummy variable to the basetable
basetable = pd.concat([basetable, dummies_country], axis=1)

# Delete the original variable from the basetable
del basetable["country"]
print(basetable.head())

In [ ]:
"""
Winsorization to handle outliers
example below to 5% highest amounts are replaced by the upper 5% percentile value and top 1 percentile to a fix value
"""
from scipy.stats.mstats import winsorize
basetable["variable_winsorized"] = 
     winsorize(
       basetable["variable"], 
       limits = [0.05,0.01])

"""
Standard deviation  to handle outliers
example below to mean+/- 3sd to replace low/high values
"""  
mean_age = basetable["age"].mean()
sd_age = basetable["age"].std()
lower_limit = mean_age - 3*sd_age
upper_limit = mean_age + 3*sd_age
basetable["age_no_outliers"] = pd.Series(
                                    [min(max(a,lower_limit), upper_limit) 
                                     for a in basetable["age"]]
                                )

> ### Part 4. advanced
  - seasonality
     - Check for seasonality and use appropriate timeline (predict Jan 2019 then use Jan 2018 rather than May 2018)
  - Using multiple snapshots
     - if train size is small, then try to include other years' data.
  - The timegap: a month as a buffer to prepare. Data in this timegap will not be used.

